In [ ]:
import numpy as np
import strawberryfields as sf
import random
from typing import List
import tensorflow as tf

In [2]:
batch_size = 100
alphas = list(np.arange(0.0, 2.1, 0.05))
betas = list(np.arange(-8.0, 2.0, 0.1))
A = 1
A = -1

In [3]:
def create_codeword(letters: List[float], codeword_size=10) -> List[float]:
        return [random.choice(letters) for _ in range(codeword_size)]

In [4]:
codeword = create_codeword([A*alphas[15], -A*alphas[15]], codeword_size=batch_size)

In [5]:
p_1 = codeword.count(A*alphas[15])/len(codeword)
p_0 = 1 - p_1

In [6]:
len(betas)


100

In [7]:
len(codeword)

100

In [8]:
eng = sf.Engine(backend="fock", backend_options={
        "cutoff_dim": 7,
        "batch_size": len(codeword),
    })

In [9]:
circuit = sf.Program(1)

beta = circuit.params("beta")
alpha = circuit.params("alpha")

with circuit.context as q:
    sf.ops.Dgate(alpha, 0.0) | q[0]
    sf.ops.Dgate(beta, 0.0) | q[0]

In [18]:
results = eng.run(circuit, args={
            "beta": betas[14],
            "alpha": codeword[14]
        })
        
# get the probability of |0>
p_zero = results.state.fock_prob([0])

# get the porbability of anything by |0>
p_one = 1 - p_zero

In [19]:
p_zero

8.120673285956007e-37

In [20]:
p_one

1.0

In [7]:
eng.reset()
prog = sf.Program(1)

alpha = prog.params("alpha")

with prog.context as q:
    sf.ops.Dgate(alpha) | q

# Assign our TensorFlow variables, so that we can
# refer to them later when differentiating/training.
a = tf.Variable(0.43)

with tf.GradientTape() as tape:
    # Here, we map our quantum free parameter `alpha`
    # to our TensorFlow variable `a` and pass it to the engine.

    result = eng.run(prog, args={"alpha": a})
    state = result.state

    # Note that all processing, including state-based post-processing,
    # must be done within the gradient tape context!
    mean, var = state.mean_photon(0)

# test that the gradient of the mean photon number is correct

grad = tape.gradient(mean, [a])
print("Gradient:", grad)

Gradient: [<tf.Tensor: shape=(), dtype=float32, numpy=0.85999966>]


In [8]:
print("Exact gradient:", 2 * a)
print("Exact and TensorFlow gradient agree:", np.allclose(grad, 2 * a))

Exact gradient: tf.Tensor(0.86, shape=(), dtype=float32)
Exact and TensorFlow gradient agree: True


In [2]:
prog = sf.Program(2)
# we can create symbolic parameters one by one
alpha = prog.params("alpha")

# or create multiple at the same time
theta_bs, phi_bs = prog.params("theta_bs", "phi_bs")

with prog.context as q:
    # States
    sf.ops.Coherent(alpha) | q[0]
    # Gates
    sf.ops.BSgate(theta_bs, phi_bs) | (q[0], q[1])
    # Measurements
    sf.ops.MeasureHomodyne(0.0) | q[0]

In [3]:
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 7})

In [4]:
mapping = {"alpha": tf.Variable(0.5), "theta_bs": tf.constant(0.4), "phi_bs": tf.constant(0.0)}

result = eng.run(prog, args=mapping)

2021-10-14 08:11:00.817102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-14 08:11:00.823202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-14 08:11:00.823794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-14 08:11:00.824731: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
print(result.samples)

tf.Tensor([[-0.27430275+0.j]], shape=(1, 1), dtype=complex64)


In [6]:
state = result.state
print("Density matrix element [0,0,1,2]:", state.dm()[0, 0, 1, 2])

Density matrix element [0,0,1,2]: tf.Tensor((0.0050318697-1.2697814e-14j), shape=(), dtype=complex64)


In [9]:
# run simulation in batched-processing mode
batch_size = 3
prog = sf.Program(1)
eng = sf.Engine("tf", backend_options={"cutoff_dim": 15, "batch_size": batch_size})

x = prog.params("x")

with prog.context as q:
    sf.ops.Thermal(x) | q[0]

x_val = tf.Variable([0.1, 0.2, 0.3])
result = eng.run(prog, args={"x": x_val})
print("Mean photon number of mode 0 (batched):", result.state.mean_photon(0)[0])

Mean photon number of mode 0 (batched): tf.Tensor([0.09999998 0.19999997 0.29999998], shape=(3,), dtype=float32)


In [10]:
# initialize engine and program objects
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 7})
circuit = sf.Program(1)

tf_alpha = tf.Variable(0.1)
tf_phi = tf.Variable(0.1)

alpha, phi = circuit.params("alpha", "phi")

with circuit.context as q:
    sf.ops.Dgate(alpha, phi) | q[0]

opt = tf.keras.optimizers.Adam(learning_rate=0.1)
steps = 50

for step in range(steps):

    # reset the engine if it has already been executed
    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        # execute the engine
        results = eng.run(circuit, args={"alpha": tf_alpha, "phi": tf_phi})
        # get the probability of fock state |1>
        prob = results.state.fock_prob([1])
        # negative sign to maximize prob
        loss = -prob

    gradients = tape.gradient(loss, [tf_alpha, tf_phi])
    opt.apply_gradients(zip(gradients, [tf_alpha, tf_phi]))
    print("Probability at step {}: {}".format(step, prob))

AttributeError: module 'keras.engine.base_layer' has no attribute 'BaseRandomLayer'